In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


batch_size = 200
learning_rate = 0.01
epochs = 10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

w1, b1 = torch.randn(200, 784, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w2, b2 = torch.randn(200, 200, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w3, b3 = torch.randn(10, 200, requires_grad=True),\
         torch.zeros(10, requires_grad=True)

#如果不初始化 会出现loss一直不变情况 也就是出现gradient vanish现象也就是梯度一直接近于0  首先出现loss的情况有以下几种：
#1.learning rate过大 2.出现gradient vanish问题，而gradient vanish主要是选择的active function和parameter有问题导致，而这里是初始化问题 
#这里的初始化是高斯分布初始化
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)


def forward(x):
    x = x@w1.t() + b1
    x = F.relu(x)
    x = x@w2.t() + b2
    x = F.relu(x)
    x = x@w3.t() + b3
    #x = F.relu(x)  输出不需要加激活函数  否者会导致loss值不更新
    return x


optimizer = optim.Adam([w1, b1, w2, b2, w3, b3], lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)

        logits = forward(data)
        loss = criterion(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data)
        test_loss += criterion(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 3.373774
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.221270
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.155372

Test set: Average loss: 0.0008, Accuracy: 9491/10000 (95%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.142992
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.153083
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.201814

Test set: Average loss: 0.0007, Accuracy: 9592/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.127641
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.118842
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.153347

Test set: Average loss: 0.0007, Accuracy: 9607/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.180492
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.254479
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.174484

Test set: Average loss: 0.0005, Accuracy: 9675/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.119898
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.090773
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.115901

Test set: Average loss: 0.0007